In [3]:
!head train.csv

qid	title
0	非法本司法考试怎么复习？
1	男友性欲太强是一种怎样的体验?
2	我国的蒙古族同胞是如何看待外蒙古国的？
3	把游戏里没有聚能设计的C4炸药放在现实中，真的能炸坦克吗？
4	如何评价郎咸平不但从空姐小三那里要回了房子，而且让小三倒赔900万？
5	深度学习相关的职位面试时一般会问什么？会问一些传统的机器学习算法吗？
6	大家有兴趣来预测及描绘腾讯的最终产品形态吗？
7	有哪些不论你再难过，听后也会笑的笑话？
8	对镜头设计而言，法兰距是短好还是长好？


In [4]:
!head sample_submit.csv

qid,topic
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8


In [1]:
import pandas as pd
import numpy as np
import jieba
from tqdm import tqdm

train = pd.read_csv('train.csv', sep='\t')
train

,qid,title
0,0,非法本司法考试怎么复习？
1,1,男友性欲太强是一种怎样的体验?
2,2,我国的蒙古族同胞是如何看待外蒙古国的？
3,3,把游戏里没有聚能设计的C4炸药放在现实中，真的能炸坦克吗？
4,4,如何评价郎咸平不但从空姐小三那里要回了房子，而且让小三倒赔900万？
...,...,...
75115,75115,如何在嘈杂的都市中像村上春树笔下的人物一样生活？
75116,75116,万圣节聚会怎样打扮会比较好玩？
75117,75117,庞中华的硬笔书法水平如何？
75118,75118,琅琊榜中有哪些细思极恐的细节？


In [18]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from sentence_transformers import SentenceTransformer
sentences_1 = ["样例数据-1", "样例数据-2"]
model = SentenceTransformer('BAAI/bge-small-zh-v1.5')
embeddings_1 = model.encode(sentences_1, normalize_embeddings=True, show_progress_bar=True)

/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
embeddings_1 = model.encode(train['title'].values, normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/2348 [00:00<?, ?it/s]

In [4]:
from sklearn.cluster import MiniBatchKMeans

# sse = []
# for n_clusters in tqdm(range(100, 10000, 500)):
#     km = MiniBatchKMeans(n_clusters=n_clusters, verbose=0, max_iter=1000)
#     km.fit(embeddings_1)
#     sse.append(km.inertia_)

In [5]:
# import matplotlib.pyplot as plt
# plt.plot(
#     list(range(100, 10000, 500))[:18],
#     sse
# )

In [21]:
km = MiniBatchKMeans(n_clusters=3000, verbose=0, max_iter=2000)
km.fit(embeddings_1)

MiniBatchKMeans(max_iter=2000, n_clusters=3000)

In [22]:
km.inertia_

31386.64453125

In [7]:
train['topic'] = km.labels_

In [8]:
train[['qid', 'topic']].to_csv('submit.csv', index=None)

In [23]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('/home/lyz/hf-models/bge-m3/') 

sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.", 
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

----------using 2*GPUs----------
[[0.626  0.3477]
 [0.3499 0.678 ]]


In [24]:
embeddings_1 = model.encode(list(train['title'].values), batch_size=100)['dense_vecs']

Inference Embeddings: 100%|██████████| 376/376 [04:10<00:00,  1.50it/s]


In [25]:
from sklearn.preprocessing import normalize
embeddings_1 = normalize(embeddings_1)

In [28]:
km = MiniBatchKMeans(n_clusters=2500, verbose=0, max_iter=2000)
km.fit(embeddings_1)
train['topic'] = km.labels_
train[['qid', 'topic']].to_csv('submit.csv', index=None)